# Finale Version mit allen Funktionen

In [2]:
import cv2
import numpy as np
import math

def GetEyes():
    # Umwandeln in Graustufenbild
    gray = cv2.cvtColor(img_without_alpha, cv2.COLOR_BGR2GRAY)

    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    print("Augen: ",eyes)
    return eyes

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngleAndSpacing(cords):
    print("Cords der Augen: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print("dela X Augen:", deltaX, "delta Y Augen: ", deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan))
    spacing = math.sqrt((deltaX ** 2 + deltaY ** 2))
    print("Abstand der Augen: ", spacing)
    return alpha, spacing

#Rotieren und Skalieren des Bildes mit berechnetem Winkel 
def RotateAndScale(image, left, right):
    deltaAlpha = right[0] - left[0]
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, deltaAlpha, 1.0)
    image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

    #Bild Sklaieren auf den gleichen Augenabstand des linken Bildes
    scale_percent = 100 - (1-left[1]/right[1])*100 #scale des Bildes in % 
    print("Skalierung in Prozent", scale_percent)
    width = int(image.shape[1] * scale_percent / 100) #Berechung der Breite
    height = int(image.shape[0] * scale_percent / 100) #Berechnung der Höhe
    dim = (width, height) #Tupel aus neuer Skalierung
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #Resize Methode
    cv2.imwrite("Result.png", result)
    return result

#Überlappen der Beiden Bilder, damit die Augen aufeinander liegen
def Overlap(image1, image2):
    #Größe Breite und Größte Höhe herausfinden
    if image1.shape[0] < image2.shape[0]:
        y = image2.shape[0]
    else:
        y = image1.shape[0]

    if image1.shape[1] < image2.shape[1]:
        x = image2.shape[1]
    else:
        x = image1.shape[1]

    print("X Finale Image: ", x, " Y Finale Image: ", y)
    #Neues np Array anlegen, dass die größte Breite und die Größte Höhe hat
    result = np.zeros((y, x, 3), dtype=np.uint8)

    #Offset mit dem das GesichtFrau Bild verschoben werden soll
    xOffset = 427
    yOffset = 239

    #Durchiterrieren aller Pixel um das Finale Bild generieren zu lassen result ist ein Bild mit RGB Kanal
    for i in range(x):
        for j in range(y):
            #try fängt indizierungen auserhalb des erlaubten Bereichs ab
            try:
                if image2[j][i][3] == 0 and j-yOffset < 0 and i-xOffset < 0:
                    result[j][i]  = [image1[j-yOffset][i-xOffset][0],image1[j-yOffset][i-xOffset][1],image1[j-yOffset][i-xOffset][2]]

                else:
                    result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
            except:
                result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
    return result


# Laden der beiden Bilder
image1 = cv2.imread('GesichtFrau.jpeg')
image2 = cv2.imread('CutFace.png')

# Erstellen der neuen Ausgabe-Matrix
img = np.zeros((max(image1.shape[0], image2.shape[0]), image1.shape[1]+image2.shape[1], 3), dtype='uint8')

# Einfügen des ersten Bildes
img[0:image1.shape[0], 0:image1.shape[1]] = image1

# Einfügen des zweiten Bildes
img[0:image2.shape[0], image1.shape[1]:image1.shape[1]+image2.shape[1]] = image2

# Überprüfen, ob das Bild erfolgreich eingelesen wurde
if img is None:
    print('Fehler: Konnte das Bild nicht einlesen')
    exit()

# Überprüfen, ob das Bild einen Alpha-Kanal hat
if len(img.shape) == 3 and img.shape[2] == 4:
    print("Das Bild hat einen Alphakanal")
    alpha = img[:, :, 3]
    img_without_alpha = img[:, :, :3]
else:
    print("Das Bild hat keinen Alphakanal")
    alpha = None
    img_without_alpha = img

#Augen Position erfassen
eyes = GetEyes()

#Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
eyesPos = list()
for i in range(len(eyes)):
    eyesPos.append((eyes[i][0],eyes[i][1]))

print("Augen Positionen von rechts nach links",sorted(eyesPos))

left = CalcAngleAndSpacing(eyesPos[:2])
print("Augen Bild Links (Winkel, Abstand der Augen)",left)
angleLeft = left[0]
spacingLeft = left[1]
right = CalcAngleAndSpacing(eyesPos[2:])
print("Augen Bild Links (Winkel, Abstand der Augen)",right)
angleRight = right[0]
spacingRight = right[1]

print("Angle Left: ",angleLeft," Angle Rigth: ", angleRight)

#CutFace nun mit Alphakanal laden
image2 = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)

#Rechtes Bild auf den Winkel drehen und Skalieren des linken Bildes
rotatet = RotateAndScale(image2, left, right)

#Bilder übereinander schieben
finalImage = Overlap(image1, rotatet)
blurFinalImage = cv2.blur(finalImage,(5,5))
cv2.imshow('Final', blurFinalImage)

# Zeichnen von Rechtecken um die erkannten Augen
for (x, y, w, h) in eyes:
    cv2.rectangle(img_without_alpha, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Zusammenführen von Alpha-Kanal und Bildoutput
# Wenn Bild Alphakanal hat
if alpha is not None:
    result = np.dstack((img_without_alpha, alpha))
# Wenn Bild keinen Alphakanal hat
else:
    result = img_without_alpha

# Anzeigen des Ergebnisses
cv2.imshow('Augen erkannt', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


Das Bild hat keinen Alphakanal
Augen:  [[ 95  38  44  44]
 [ 47  61  35  35]
 [534 106  47  47]
 [594 109  45  45]]
Augen Positionen von rechts nach links [(47, 61), (95, 38), (534, 106), (594, 109)]
Cords der Augen:  [(95, 38), (47, 61)]
dela X Augen: 48 delta Y Augen:  -23
Abstand der Augen:  53.225933528685054
Augen Bild Links (Winkel, Abstand der Augen) (-29.768142962234492, 53.225933528685054)
Cords der Augen:  [(534, 106), (594, 109)]
dela X Augen: -60 delta Y Augen:  -3
Abstand der Augen:  60.07495318350236
Augen Bild Links (Winkel, Abstand der Augen) (2.8671786895428357, 60.07495318350236)
Angle Left:  -29.768142962234492  Angle Rigth:  2.8671786895428357
Skalierung in Prozent 88.59920933455147
X Finale Image:  542  Y Finale Image:  361


QObject::moveToThread: Current thread (0x251d350) is not the object's thread (0x3260a80).
Cannot move to target thread (0x251d350)

QObject::moveToThread: Current thread (0x251d350) is not the object's thread (0x3260a80).
Cannot move to target thread (0x251d350)

QObject::moveToThread: Current thread (0x251d350) is not the object's thread (0x3260a80).
Cannot move to target thread (0x251d350)

QObject::moveToThread: Current thread (0x251d350) is not the object's thread (0x3260a80).
Cannot move to target thread (0x251d350)

QObject::moveToThread: Current thread (0x251d350) is not the object's thread (0x3260a80).
Cannot move to target thread (0x251d350)

QObject::moveToThread: Current thread (0x251d350) is not the object's thread (0x3260a80).
Cannot move to target thread (0x251d350)

QObject::moveToThread: Current thread (0x251d350) is not the object's thread (0x3260a80).
Cannot move to target thread (0x251d350)

QObject::moveToThread: Current thread (0x251d350) is not the object's thread